In [1]:
import numpy
import pandas as pd
import xlwt
from xlwt import Workbook
from __future__ import print_function, division
from data_loader import ImageFolder
import torch.nn.functional as F
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import torchvision.utils as utils

In [208]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])

data_transforms = transforms.Compose([transforms.Resize(224), transforms.CenterCrop(224), 
                                      transforms.ToTensor(), normalize ])

data_dir = '/Users/suchethas/Documents/hymenoptera_data/train'
image_datasets = ImageFolder(data_dir, transform=data_transforms)
dataloaders = torch.utils.data.DataLoader(image_datasets, batch_size = 10, shuffle = True, num_workers=4)

res_dir = 'results_auto'

In [43]:
#model = vgg16(pretrained=False)

#Removing the classifier layer
#modified_model = nn.Sequential(*list(model.features.children()))


#model.load_state_dict(torch.load('/Users/suchethas/Downloads/model_antsbees.pth'))
#model.eval()

# criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)  

In [209]:
def make_layers_encoder(cfg, batch_norm=False):
    layers = []
    in_channels = 3
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2, return_indices=True)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)

def make_layers_decoder(cfg_decoder, batch_norm=False):
    layers_decoder = []
    inn_channels = 512
    for v in cfg_decoder:
        if v == 'M':
            layers_decoder += [nn.MaxUnpool2d(kernel_size=2, stride=2)]
        else:
            trans_conv2d = nn.ConvTranspose2d(inn_channels, v, kernel_size=3, padding=1)
            if batch_norm:
                layers_decoder += [trans_conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers_decoder += [trans_conv2d, nn.ReLU(inplace=True)]
            inn_channels = v
    return nn.Sequential(*layers_decoder)

cfg_decoder = {
    #'D' : ['M', 512, 512, 512, 'M', 512, 512,  256, 'M', 256, 256,  128, 'M', 128,  64, 'M', 64, 3],
    'A' : ['M', 512, 512, 'M', 512, 256, 'M', 256, 128, 'M', 64, 'M', 3]
    #'A' : [512, 512, 512, 256, 256, 128, 64, 3]
}

cfg = {
    'A': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'B': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'D': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'E': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}


class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder,self).__init__()
        
        self.encoder = make_layers_encoder(cfg['A'])
        self.decoder = make_layers_decoder(cfg_decoder['A'])
        
    def forward(self,x):
        ind = []
        x = self.handlesequential_encoder(self.encoder,x,ind)
        x = self.handlesequential_decoder(self.decoder,x,ind)
        return x


    def handlesequential_encoder(self,layer,x,ind):
        for i,module in enumerate(layer.children()):
            if type(module) ==nn.MaxPool2d:
                x, inde1 = module(x)
                #print(x.shape, inde1.shape)
                ind.append(inde1)
            else:
                x = module(x)
        return x
    
    def handlesequential_decoder(self,layer,x,ind):
        for i,module in enumerate(layer.children()):
            if type(module) ==nn.MaxUnpool2d:
                x = module(x, ind.pop())
            else:
                x = module(x)
        return x
    
ae = Autoencoder()


In [204]:
def create_path(path):
    if os.path.isdir(path) is False:
        os.makedirs(path)

In [207]:
#criterion = nn.CrossEntropyLoss()
criterion = nn.MSELoss()
optimizer = optim.SGD(ae.parameters(), lr=0.01, momentum=0.9)  

for epoch in range(1):
    print("EPOCH:", epoch)
    for batch, inp in enumerate(dataloaders,1):
        print("batch: ", batch)
        inputs = inp[0]
        targets = inp[1]
        paths = inp[2]

        inputs.requires_grad = True
        outputs = ae(inputs)
        
        with torch.no_grad():
            for xi in range(10):
                save_path = os.path.join(res_dir, paths[xi].split(data_dir + '/')[1].split('.')[0])
                create_path(os.path.join(save_path.split('/')[0], save_path.split('/')[1]))
                utils.save_image(inputs[xi,:,:,:], save_path + '_orig.png',  normalize=True, padding=0)
                utils.save_image(outputs[xi,:,:,:], save_path + '_auto.png',  normalize=True, padding=0)

        loss = criterion(outputs, inputs)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print('loss:{:.4f}'.format(loss.item()))
    

EPOCH: 0
batch:  1
torch.Size([10, 3, 224, 224]) torch.Size([10, 3, 224, 224])
loss:0.2651
batch:  2
torch.Size([10, 3, 224, 224]) torch.Size([10, 3, 224, 224])
loss:0.2250
batch:  3
torch.Size([10, 3, 224, 224]) torch.Size([10, 3, 224, 224])
loss:0.2132
batch:  4
torch.Size([10, 3, 224, 224]) torch.Size([10, 3, 224, 224])


Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/anaconda3/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/anaconda3/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/anaconda3/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/anaconda3/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/anaconda3/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/anaconda3/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/anaconda3/lib/python3.6/multiprocessing/connec

KeyboardInterrupt: 

  File "/anaconda3/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/anaconda3/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/anaconda3/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/anaconda3/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/anaconda3/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
  File "/anaconda3/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
  File "/anaconda3/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
  File "/anaconda3/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
BrokenPipeError:

In [211]:
print(ae)

Autoencoder(
  (encoder): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (16): Conv2d(512, 51

In [199]:
data_dir_test = '/Users/suchethas/Documents/hymenoptera_data/val'
image_datasets_test = ImageFolder(data_dir_test, transform=data_transforms)
testloaders = torch.utils.data.DataLoader(image_datasets_test, batch_size = 10, shuffle = True, num_workers=4)

for batch, inp in enumerate(testloaders,1):
    print("batch: ", batch)
    inputs = inp[0]
    targets = inp[1]
    paths = inp[2]

    
    outputs = ae(inputs)
    
    
    for xi in range(10):
        save_path = os.path.join(res_dir, paths[xi].split(data_dir_test + '/')[1].split('.')[0])
        create_path(os.path.join(save_path.split('/')[0], save_path.split('/')[1]))
        utils.save_image(inputs[xi,:,:,:], save_path + '_orig.png', normalize=True, padding=0)
        utils.save_image(outputs[xi,:,:,:], save_path + '_auto.png', normalize=True, padding=0)

batch:  1
batch:  2
batch:  3
batch:  4
batch:  5
batch:  6
batch:  7
batch:  8
batch:  9
batch:  10
batch:  11
batch:  12
batch:  13
batch:  14
batch:  15
batch:  16


IndexError: tuple index out of range